In [1]:
import healpy
import numpy as np
from astropy.coordinates import SkyCoord
from dustmaps.sfd import SFDQuery

from multiorder import AbstractMultiorderMapBuilder
from min_max_mean_state import MinMaxMeanState, MinMaxMeanStateMerger
from paths import *

In [8]:
class StateMerger(MinMaxMeanStateMerger):
    def __init__(self, threshold: float = 0.16):
        self.threshold = threshold
    
    def validate_state(self, state: MinMaxMeanState) -> bool:
        norm = max(map(abs, [state.min, state.max]))
        if norm == 0.0:
            return True
        return (state.max - state.min) / norm <= self.threshold
    

def ebv(norder: int):
    n_size = healpy.order2nside(norder)
    n_pix = healpy.order2npix(norder)
    
    indexes = np.arange(n_pix)
    ra, dec = healpy.pix2ang(n_size, indexes, nest=True, lonlat=True)
    coord = SkyCoord(ra=ra, dec=dec, unit='deg')
    
    sfd_query = SFDQuery(INPUT_DIR)
    return sfd_query(coord)
    

class Builder(StateMerger, AbstractMultiorderMapBuilder):
    def __init__(self, max_norder, threshold):
        AbstractMultiorderMapBuilder.__init__(self, max_norder)
        StateMerger.__init__(self, threshold)
        
        self.ebv = ebv(max_norder)
        
    def calculate_state(self, index_max_norder: int) -> MinMaxMeanState:
        value = self.ebv[index_max_norder].item()
        return MinMaxMeanState(min=value, max=value, mean=value)

In [9]:
%%time

tiles = Builder(max_norder=3, threshold=0.9).build()

CPU times: user 23.2 ms, sys: 18.6 ms, total: 41.8 ms
Wall time: 51.5 ms


In [20]:
%%time

tiles = Builder(max_norder=11, threshold=0.016).build()

CPU times: user 1min 7s, sys: 2.6 s, total: 1min 10s
Wall time: 1min 11s
